In [1]:
import os
from datasets import load_dataset
path = os.path.join("..","dataset","tiny_stories","en-zh")
data = load_dataset("json",data_files={"train":os.path.join(path,"train.jsonl"),"test":os.path.join(path,"valid.jsonl")})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import MT5Tokenizer
from transformers import RobertaForSequenceClassification,RobertaTokenizer,get_linear_schedule_with_warmup
from datasets import load_dataset
from torch.utils.data import Dataset
import os
import random
import numpy as np
import evaluate
import torch
import argparse
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.functional import F
from torch.cuda.amp import autocast as autocast,GradScaler
import pandas as pd
from sklearn.metrics import f1_score
from torch.optim import AdamW
import bleu


In [ ]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
tokenizer.model_max_length = 512

In [ ]:
from transformers import MT5ForConditionalGeneration,Seq2SeqTrainingArguments,Seq2SeqTrainer

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")

In [ ]:
def preprocess_function(examples):
    inputs = ["Translate English into  Chinese:" + example['text'] for example in examples['src_text']]
    tgts = [ example for example in examples['tgt_text'] ]
    model_inputs = tokenizer(inputs,text_target=tgts,padding=True,truncation=True)  
    return model_inputs

In [ ]:
tokenzied_data = data.map(preprocess_function,batched=True,remove_columns=['src_text','tgt_text'])

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(tokenzied_data['train'][0]['input_ids'])

'Translate English into Chinese:Once upon a time, there was a little girl named Lily. She loved to talk to her friends, and she loved to play with her toys. One day, she found a lemon in her kitchen. Lily thought the lemon was a toy, so she took it to her room to play with it. Lily started to feel helpless because the lemon was not fun like her other toys. She tried to talk to the lemon, but it did not talk back. Lily was sad and didn\'t know what to do. Then, something unexpected happened. The lemon started to grow bigger and bigger! Suddenly, the lemon turned into a big, friendly lemon man. He could talk, and he was not helpless at all! He said, "Hi Lily, I am Mr. Lemon. I was hiding in your kitchen, and now I am here to play with you." Lily was so happy and surprised. They played together all day long, and Lily had a new, fun friend.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [ ]:
tokenizer.decode(tokenzied_data['train'][0]['labels'])

'从前,有一个叫莉莉的小女孩。她喜欢和她的朋友聊天,她也喜欢玩她的玩具。一天,她在厨房里发现了一个柠檬。莉莉以为柠檬是玩具,所以她把它带到房间里玩。莉莉开始感到无助,因为柠檬不像她的其他玩具那样有趣。她试着和柠檬说话,但它不顶嘴。莉莉很难过,不知道该怎么办。然后,意想不到的事情发生了。柠檬开始变得越来越大!突然,柠檬变成了一个又大又友好的柠檬人。他会说话,一点也不无助!他说:“嗨,莉莉,我是柠檬先生。我躲在你的厨房里,现在我来和你一起玩。”莉莉既高兴又惊讶。他们整天都在一起玩,莉莉有了一个有趣的新朋友。</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
tokenzied_data['test']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})

In [ ]:
def collate_fn(examples):
    return tokenizer.pad(examples, padding="max_length", return_tensors="pt")

In [ ]:
train_dataloader = DataLoader(tokenzied_data['train'],shuffle=True,collate_fn=collate_fn,batch_size = 4)
eval_dataloader = DataLoader(tokenzied_data['test'] , collate_fn=collate_fn,batch_size = 2)


In [ ]:
num_epochs = 5
lr = 1e-5
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)


In [34]:
device = "cuda"
model.to(device)
max_eval_codebleu = 0
iter_to_accumlate = 4
epochloss = []
for epoch in range(num_epochs):
    model.train()
    allloss = 0
    for step,batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss/iter_to_accumlate
        loss.backward()
        allloss += loss.item()
        epochloss.append(loss.item())
        if (step+1)%iter_to_accumlate==0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        if (step+1)%(4*iter_to_accumlate) == 0:
            print("epoch",epoch,"step",step,"loss",loss,sep=" ")
    print("epoch",epoch,"trainLoss:",allloss/(len(train_dataloader)*4))
    count = 0
    model.eval()
    validloss = []
    all_me = []
    for step,batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            output = model.generate(**batch)
        # validloss += [output.loss.item()]
        references = []
        hypothesis = []
        label_strs = tokenizer.batch_decode(batch['labels'].cpu(),skip_special_tokens = True)
        output_strs = tokenizer.batch_decode(output.cpu())
        for i in range(len(label_strs)):
            all_me.append(bleu.sentence_bleu(references= [label_strs[i].split()], hypothesis=output_strs[i].split()))
        print(label_strs,output_strs,all_me)
    eval_bleu = sum(all_me)/len(all_me)
    print("epoch ",epoch,"bleu ",eval_bleu)
    if eval_bleu > max_eval_codebleu:
        min_eval_codebleu = eval_bleu
        model.save_pretrained("Jchew/mT5ForTranslate_English2Chinese")
        model.push_to_hub("Jchew/mT5ForTranslate_English2Chinese")
        tokenizer.push_to_hub("Jchew/mT5ForTranslate_English2Chinese")

  0%|          | 0/1000 [00:00<?, ?it/s]/home/ljc/anaconda3/envs/adv/lib/python3.8/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 2/1000 [00:00<02:58,  5.59it/s]

['从前有一辆跑车,名叫斯皮迪。他喜欢开快车。一天,他在一座大山附近的路上行驶。他看到一块牌子上写着“危险的路”。但是斯皮迪不在乎。他想走得快。当斯皮迪上山时,他看到一只名叫伊万的慢乌龟。伊万说:“斯皮迪,这条路很危险。你不应该在这里走得快。”斯皮迪笑着说:“我不害怕。我是一辆跑车。”斯皮迪快速上山。但是这条路非常危险。有大石头和洞。斯皮迪差点掉进一个洞里。他很害怕。他停下来说:“伊万是对的。我在这条路上应该小心。”斯皮迪回到山下去找伊万。他说:“谢谢你警告我。你是个好朋友。”然而笑着说:“不客气,斯皮迪。我们应该在危险的道路上永远小心。”从那天起,斯皮迪和然而是最好的朋友。他们总是互相帮助保持安全。斯皮迪学会了在危险的道路上小心。', '从前,在一个小房子里,有一个善良的女孩,名叫莉莉。她有一面魔镜。每天,她都会选择一个新朋友在镜子里玩。一天,她选择了一只猫。猫很好,他们玩得很开心。第二天,莉莉选择了一只狗在镜子里玩。他们玩了一个球,玩得很开心。但是后来,意想不到的事情发生了。前一天的猫回到了镜子里。猫和狗开始打架。莉莉看到她的朋友打架很难过。她要求他们善待彼此。猫和狗停止了打架,看着莉莉。他们为让她难过而感到难过。然后,猫和狗成了朋友。他们都在镜子里一起玩耍,度过了快乐的一天。'] ['<pad> <extra_id_0>. Retrieved from "https://en. <extra_id_1>.cn/"</s>', '<pad> <extra_id_0>. They all had fun.</s><pad><pad><pad><pad><pad><pad><pad><pad>'] [0, 0]
['从前,有一个叫露西的小女孩。她有一本特别的书,叫做日记。在这本日记里,她写了每天发生的事情。她也喜欢画画。日记是她最好的朋友。一天,露西发现了一块罕见的漂亮的石头。它闪亮光滑。她太高兴了!她把石头带回家,给她的父母看。他们也很开心。露西想保护石头的安全,所以她把它放在日记里。每天晚上,在她睡觉之前,露西都会打开日记,看看这块罕见的石头。她会写下她的一天,并画一张石头的照片。她的日记现在更加特别了。这是一个快乐事情发生的地方。露西和她的日记从此幸福地生活在一起。', '一天,一个叫蒂姆的小男孩在他的院子里发现了一个灯泡。他认为这是一个愚蠢的玩具。他把它给他

  0%|          | 4/1000 [00:00<02:43,  6.11it/s]

['一天,一个叫米娅的年轻女孩和她的妈妈在树林里散步。他们走着走着,米娅听到灌木丛里有轻微的沙沙声。米娅的妈妈跪了下来,小声说:“让我们非常安静,米娅。”米娅很好奇,试图仔细看看,但她的妈妈把她拉了回来。从灌木丛里,一位女士走了出来。她穿着一件漂亮的绿色连衣裙,走得很慢。女士轻声说。“你准备好我将要揭示的事情了吗?”米娅的妈妈点了点头,但米娅很困惑。女士微笑着,慢慢地张开了手。里面是一块闪闪发光的蓝色石头。“这是给你的,米娅,”她说。米娅非常兴奋,立刻抓住了它。“这是森林的秘密,”女士解释道。“我把它给你,因为我认为你已经准备好了。”弥雅抬头看着母亲,不知道该怎么办。母亲安慰地笑了笑。弥雅拥抱着女士,感激森林的秘密。女士眼睛闪烁着光芒,说:“我们保守秘密,好吗?”弥雅点点头,已经梦想着带着她的秘密去冒险。', '从前,在一个小镇上,有一只叫基蒂的可怜猫。基蒂非常难过,因为其他动物会取笑她。他们会说:“你是一只可怜的猫!你没有像我们这样漂亮的家!”一天,基蒂走在一口井边。她看见一只叫鲍比的小鸟坐在井边。鲍比也很难过,因为其他鸟也会取笑他。“我这么小,他们嘲笑我,”鲍比说。基蒂和鲍比成了朋友。他们决定互相帮助,不让取笑让他们难过。他们在井边玩耍,一起玩得很开心。很快,其他动物看到他们有多开心,就不再取笑他们了。基蒂和鲍比很开心,他们一起住在井边。'] ["<pad> <extra_id_0>, but her mother couldn't.</s>", '<pad> <extra_id_0>, and they laughed.</s><pad><pad>'] [0, 0, 0, 0, 0, 0]
['从前,有一个叫莎莉的小女孩。她喜欢在外面玩。一天,天开始下雨了。雨下得很大,弄出了很大的水坑。莎莉很难过,因为她不能在外面玩。莎莉的妈妈看到她看起来很难过。她说:“莎莉,我们今天为什么不接受雨水,在里面玩呢?”莎莉想了一会儿,然后笑了。她说:“好的,妈妈!我们在里面玩吧!”莎莉和她妈妈整天玩游戏,看书。他们玩得很开心!雨停了,太阳出来了。莎莉向外面看,看到了一道很大的彩虹。她很高兴自己接受了雨水,和妈妈一起度过了愉快的一天。', '一天,一个叫汤姆的小男孩去拜访他的朋友——鳄鱼。鳄鱼住在一个大湖里。汤姆和鳄鱼喜欢一起玩耍和聊天。一天,当汤姆去看鳄鱼时,他看到了一件非常悲

  1%|          | 6/1000 [00:01<02:36,  6.35it/s]

...

  3%|▎         | 28/1000 [00:04<02:32,  6.36it/s]

['从前,在一所小房子里,有一只小狗,名叫蒂尼。蒂尼喜欢玩耍和弄脏。一天,蒂尼出去发现了一个大泥坑。他跳进去玩了一整天。他非常开心,玩得很开心。当蒂尼回家时,他妈妈看到了他,说:“蒂尼,你真脏!你需要马上洗澡!”蒂尼不喜欢洗澡,但他知道他妈妈是对的。所以,他去了浴室,准备洗澡。蒂尼试图找到一种让洗澡变得有趣的方法,但他就是不喜欢。他溅了溅水,玩了玩他的玩具,但他仍然感到悲伤。洗澡结束时,蒂尼很干净,但他想念泥坑。他希望他能回去再玩一次。', '从前,有一个王子。他是一个多毛的王子。他住在一所大房子里。王子有很多朋友。他们喜欢和他一起玩。一天,王子遇到了一个女孩。她人很好。她想给他看点东西。王子很高兴。他想看看她要展示什么。女孩带王子去了公园。公园很漂亮。有许多花。王子和女孩在公园里玩耍。他们玩得很开心。王子喜欢这个公园。他非常高兴。结束。'] ['<pad> <extra_id_0>, but he did not like it.</s>', '<pad> <extra_id_0>. Please select language.</s><pad><pad>'] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['一天,一个叫米娅的小女孩去外面玩。太阳很热,晒在她的皮肤上感觉很刺耳。她看到一个大盒子,里面有许多玩具。米娅觉得按颜色分类玩具会很有趣。当她分类玩具时,一个叫汤姆的男孩来玩了。他说:“嗨,米娅!我能帮你分类玩具吗?”米娅微笑着说:“是的,欢迎汤姆!你可以帮我。”他们一起把玩具分类成成堆的红色、蓝色、绿色和黄色。他们一起玩得很开心。当他们完成后,他们跑进去喝杯冷饮,躲避严酷的阳光。', '从前,有一个笨拙的士兵,名叫汤姆。他不像其他士兵那样擅长行军。他会经常绊倒。他的朋友们会戏弄他说:“汤姆,你需要像我们一样学习走路!”一天,汤姆在森林里散步,这时他遇到了一只小鸟。小鸟很难过,因为它不会飞。汤姆为小鸟感到难过,说:“别担心,小鸟。我会帮你学会飞的。”汤姆和小鸟每天练习。他们成了很好的

  3%|▎         | 29/1000 [00:04<02:36,  6.20it/s]


['从前,有一个很大的广告牌。它非常重要。广告牌上有一张猫的照片。每天,人们路过都会看到这只猫。他们会微笑着谈论这只猫。一天,一个叫蒂姆的小男孩看到了广告牌。他看着这只猫,认出了它。“那是我的猫,路威!”蒂姆说。他很高兴看到他的猫出现在重要的广告牌上。蒂姆回到家,告诉了他妈妈广告牌的事。他妈妈为路威感到非常自豪。他们一起去看了广告牌。他们和路威以及广告牌合了影。从此他们都过着幸福的生活。', '从前,在一个著名的大宫殿里,住着一位善良的国王和王后。他们有一个小女孩,名叫莉莉。她喜欢玩她的玩具,在宫殿里跑来跑去。一天,莉莉发现宫殿里有一扇小门。她想看看里面有什么。她试图打开它,但它被锁上了。莉莉向她的妈妈——王后——要钥匙开门。王后给了莉莉钥匙,她打开了门。在里面,她发现了一个美丽的花园,里面有鲜花和蝴蝶。莉莉在花园里玩了一整天,玩得很开心。从那天起,著名的宫殿有了一个新的、特别的地方供莉莉玩耍。'] ['<pad> <extra_id_0>, Fluffy, Fluffy</s><pad><pad><pad>', '<pad> <extra_id_0>, a palace, called.</s>'] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


KeyboardInterrupt: 